In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

In [4]:
spark = SparkSession.builder.master("local[*]").appName('test2').config("spark.jars", "/home/jovyan/work/apps/postgresql-42.7.4.jar").getOrCreate()

In [5]:
df = spark.read.parquet('fhvhv/2021/01/')

In [6]:
df.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02875|2021-01-04 09:54:16|2021-01-04 10:20:02|         152|         193|   NULL|
|           HV0003|              B02836|2021-01-03 16:45:51|2021-01-03 16:58:38|         208|         213|   NULL|
|           HV0003|              B02864|2021-01-04 15:51:39|2021-01-04 16:03:16|         145|         202|   NULL|
|           HV0003|              B02395|2021-01-02 08:15:37|2021-01-02 08:19:39|          78|         169|   NULL|
|           HV0005|              B02510|2021-01-02 06:11:21|2021-01-02 06:40:23|          17|          73|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [7]:
df.createOrReplaceTempView('data')

In [9]:
result = spark.sql('''
    SELECT 
        hvfhs_license_num, 
        COUNT(*) AS total_trips
    FROM data
    GROUP BY hvfhs_license_num
    ORDER BY total_trips DESC
''')

In [10]:
#postgres configuration
postgres_url = "jdbc:postgresql://postgres_container:5432/mydatabase"

postgres_properties = {
    "user": "admin",
    "password": "password",
    "driver": "org.postgresql.Driver"
}

In [11]:
result.write.jdbc(url=postgres_url, table="revenue_report", mode="overwrite", properties=postgres_properties)

In [12]:
spark.stop()